In [1]:
print("test")

test


설치파일

In [13]:
!pip install -U demucs
!pip install ffmpeg-python
!pip install soundfile librosa
!pip install soundfile

In [2]:
# 1️⃣ 라이브러리 로드
from demucs import pretrained
from demucs.apply import apply_model
import torchaudio
import torch
import os
#윈도우 에서는 Soundfile이 mp3를 지원하지 않아서 sox_io백엔드로 변경 필요
torchaudio.set_audio_backend("sox_io")
print("사용 가능 백엔드 목록 출력",torchaudio.list_audio_backends())


사용 가능 백엔드 목록 출력 ['soundfile']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12112\1851243015.py:8: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")


In [5]:
import torchaudio
print("백엔드 리스트:",torchaudio.list_audio_backends())

#테스트 로드
mix, sr = torchaudio.load(r'D:\혼자 하는 프젝\드럼 악보 전사\Automatic_Drum_Transcription\data\mp3\한페될.mp3')
print("샘플 shape:",mix.shape, sr)

백엔드 리스트: ['soundfile']
샘플 shape: torch.Size([2, 9156608]) 44100


In [7]:

# 2️⃣ 모델 로드 (fine-tuned)
bag = pretrained.get_model("htdemucs_ft") #BagOfModels 타입
device = "cuda" if torch.cuda.is_available() else "cpu"
bag.to(device)
bag.eval()

# 3️⃣ 분리 실행 함수
def separate_drums(input_path, output_dir='separated'):
    #오디오 불러오기
    mix, sr = torchaudio.load(input_path)
    mix = mix.to(device)

    #배치 차원 추가
    sources = apply_model(bag, mix.unsqueeze(0), device = device)
    #소스 차원 = (1, num_stems, channels, samples)
    
    #drums 인덱스 찾기
    stem_names = bag.sources # ['drums','bass','other','vocals']
    drums_index = stem_names.index('drums')
    drums = sources[0,drums_index]
    os.makedirs(output_dir, exist_ok=True)

    #결과 저장
    torchaudio.save(f"{output_dir}/drums.wav", drums.cpu(), sr)
    print(f"Drums stem 저장 완료: {output_dir}/drums.wav")

# 4️⃣ 실제 테스트
separate_drums(r'D:\혼자 하는 프젝\드럼 악보 전사\Automatic_Drum_Transcription\data\mp3\한페될.mp3')


Drums stem 저장 완료: separated/drums.wav
